# 0. Installation und Abhängigkeiten

In [8]:
!pip3 install --upgrade pip --user

  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
!pip install swig --user

In [10]:
!pip install stable_baselines3 --user

  Using cached stable_baselines3-1.6.0-py3-none-any.whl (177 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached torch-1.12.1-cp38-cp38-win_amd64.whl (161.9 MB)
  Using cached gym-0.21.0-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.26.0
    Uninstalling gym-0.26.0:
      Successfully uninstalled gym-0.26.0


In [11]:
!pip install tensorflow

In [12]:
!pip install gym  --user

In [3]:
!pip install box2d-py --user


  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp38-cp38-win_amd64.whl size=479793 sha256=5cff78debe872fc93dd7fcbd394fd33dad9dcd87291d0b17b60917a8baf99e3c
  Stored in directory: c:\users\ra\appdata\local\pip\cache\wheels\cc\4f\d6\44eb0a9e6fea384e58f19cb0c4125e46a23af2b33fe3a7e81c
Successfully built box2d-py


In [5]:
!pip install pygame

In [8]:
!pip install pyglet


     ---------------------------------------- 1.1/1.1 MB 7.2 MB/s eta 0:00:00


In [2]:
import stable_baselines3
import gym 
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 1. Testen eines zufälligen Agenten in einem zufälligen Environment

In [3]:
#environment_name = 'MountainCar-v0'
environment_name = 'MountainCarContinuous-v0'
env = gym.make(environment_name)

In [4]:
# Observation and action space 
obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

The observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
The action space: Box([-1.], [1.], (1,), float32)


In [5]:
import matplotlib.pyplot as plt 

# reset the environment and see the initial observation
obs = env.reset()
print("The initial observation is {}".format(obs))

# Sample a random action from the entire action space
random_action = env.action_space.sample()

# # Take the action and get the new observation space
new_obs, reward, done, info = env.step(random_action)
print("The new observation is {}".format(new_obs))

The initial observation is [-0.463568  0.      ]
The new observation is [-0.46500617 -0.00143816]


In [8]:
env.render(mode = "human")

True

In [7]:
env.close()

In [15]:
state = env.reset()
total_rewards = 0
for i in range(500):
    action = env.action_space.sample()+100
    #print(action)
    env.render()
    next_state,reward,done,_ = env.step(action)
    total_rewards += reward

env.close()
print(total_rewards)
    

-499915.2488844755


In [42]:
import time 

# Number of steps you run the agent for 
num_steps = 2500

obs = env.reset()

for step in range(num_steps):
    # take random action, but you can also do something more intelligent
    #action = my_intelligent_agent_fn(obs) 
    action = env.action_space.sample()
    #print('Action:{}'.format(action))
    # apply the action
    obs, reward, done, info = env.step(action)
   
    # Render the env
    env.render()

    # Wait a bit before the next frame unless you want to see a crazy fast video
    time.sleep(0.001)
    
    # If the epsiode is up, then start another one
    if done:
        print('loop')
        env.reset()

# Close the env
env.close()

loop
loop


In [9]:
type(env.observation_space)

gym.spaces.box.Box

In [10]:
print("Upper Bound for Env Observation", env.observation_space.high)
print("Lower Bound for Env Observation", env.observation_space.low)

Upper Bound for Env Observation [0.6  0.07]
Lower Bound for Env Observation [-1.2  -0.07]


In [10]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    #env.step(2)
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        #print('N-State:{} Reward:{}'.format(n_state, reward) )
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-33.526083840052436
Episode:2 Score:-33.551611845424304
Episode:3 Score:-34.40232842221686
Episode:4 Score:-32.25606349448578
Episode:5 Score:-33.45119314377286
Episode:6 Score:-34.591995192213034
Episode:7 Score:-32.87120414390912
Episode:8 Score:-32.98304973896764
Episode:9 Score:-33.36751893262781
Episode:10 Score:-31.864901204342743


# 2. Erstellen und Trainieren eines Modells

In [12]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# MlpPolicy steht für Multilayer Perceptron (2 layers of 64)
#... usw. siehe: https://stable-baselines.readthedocs.io/en/master/modules/policies.html
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [13]:
model.learn(total_timesteps=500000)

-----------------------------
| time/              |      |
|    fps             | 2240 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1492       |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00724729 |
|    clip_fraction        | 0.0254     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.37      |
|    explained_variance   | -0.00679   |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0104     |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0127    |
|    std                  | 0.915      |
|    value_loss           | 0.0752     |
----------------------------------------
-----------

-----------------------------------------
| time/                   |             |
|    fps                  | 1191        |
|    iterations           | 12          |
|    time_elapsed         | 20          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.008145383 |
|    clip_fraction        | 0.0316      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.476      |
|    explained_variance   | -0.0114     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0192     |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.0182     |
|    std                  | 0.375       |
|    value_loss           | 0.00412     |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1181         |
|    iterations           | 13           |
|    time_elapsed         | 22

------------------------------------------
| time/                   |              |
|    fps                  | 1162         |
|    iterations           | 23           |
|    time_elapsed         | 40           |
|    total_timesteps      | 47104        |
| train/                  |              |
|    approx_kl            | 0.0045728125 |
|    clip_fraction        | 0.02         |
|    clip_range           | 0.2          |
|    entropy_loss         | 0.366        |
|    explained_variance   | 0.0181       |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0129      |
|    n_updates            | 220          |
|    policy_gradient_loss | -0.0114      |
|    std                  | 0.163        |
|    value_loss           | 0.000416     |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1161        |
|    iterations           | 24          |
|    time_elaps

------------------------------------------
| time/                   |              |
|    fps                  | 1151         |
|    iterations           | 34           |
|    time_elapsed         | 60           |
|    total_timesteps      | 69632        |
| train/                  |              |
|    approx_kl            | 0.0045093438 |
|    clip_fraction        | 0.0193       |
|    clip_range           | 0.2          |
|    entropy_loss         | 1.11         |
|    explained_variance   | -0.00236     |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0141      |
|    n_updates            | 330          |
|    policy_gradient_loss | -0.00978     |
|    std                  | 0.0776       |
|    value_loss           | 8.12e-05     |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1151        |
|    iterations           | 35          |
|    time_elaps

------------------------------------------
| time/                   |              |
|    fps                  | 1146         |
|    iterations           | 45           |
|    time_elapsed         | 80           |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0039999876 |
|    clip_fraction        | 0.0161       |
|    clip_range           | 0.2          |
|    entropy_loss         | 1.79         |
|    explained_variance   | -0.00628     |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00632     |
|    n_updates            | 440          |
|    policy_gradient_loss | -0.00918     |
|    std                  | 0.0393       |
|    value_loss           | 2.19e-06     |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1146        |
|    iterations           | 46          |
|    time_elaps

...

-----------------------------------------
| time/                   |             |
|    fps                  | 1139        |
|    iterations           | 199         |
|    time_elapsed         | 357         |
|    total_timesteps      | 407552      |
| train/                  |             |
|    approx_kl            | 0.008599453 |
|    clip_fraction        | 0.171       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.16        |
|    explained_variance   | -1.7        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0249     |
|    n_updates            | 1980        |
|    policy_gradient_loss | 0.00893     |
|    std                  | 0.00138     |
|    value_loss           | 1.35e-08    |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1139         |
|    iterations           | 200          |
|    time_elapsed         | 35

-----------------------------------------
| time/                   |             |
|    fps                  | 1139        |
|    iterations           | 210         |
|    time_elapsed         | 377         |
|    total_timesteps      | 430080      |
| train/                  |             |
|    approx_kl            | 0.008681977 |
|    clip_fraction        | 0.139       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.33        |
|    explained_variance   | -0.0175     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0133      |
|    n_updates            | 2090        |
|    policy_gradient_loss | 0.0058      |
|    std                  | 0.00117     |
|    value_loss           | 1.12e-10    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1139        |
|    iterations           | 211         |
|    time_elapsed         | 379   

-----------------------------------------
| time/                   |             |
|    fps                  | 1138        |
|    iterations           | 221         |
|    time_elapsed         | 397         |
|    total_timesteps      | 452608      |
| train/                  |             |
|    approx_kl            | 0.007127316 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.41        |
|    explained_variance   | -1.13       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0169      |
|    n_updates            | 2200        |
|    policy_gradient_loss | 0.00454     |
|    std                  | 0.00108     |
|    value_loss           | 3.2e-11     |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1139       |
|    iterations           | 222        |
|    time_elapsed         | 399       

-----------------------------------------
| time/                   |             |
|    fps                  | 1139        |
|    iterations           | 232         |
|    time_elapsed         | 416         |
|    total_timesteps      | 475136      |
| train/                  |             |
|    approx_kl            | 0.018018093 |
|    clip_fraction        | 0.216       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.47        |
|    explained_variance   | -0.245      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00447     |
|    n_updates            | 2310        |
|    policy_gradient_loss | 0.0134      |
|    std                  | 0.00102     |
|    value_loss           | 4.28e-11    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1139        |
|    iterations           | 233         |
|    time_elapsed         | 418   

-----------------------------------------
| time/                   |             |
|    fps                  | 1137        |
|    iterations           | 243         |
|    time_elapsed         | 437         |
|    total_timesteps      | 497664      |
| train/                  |             |
|    approx_kl            | 0.019431444 |
|    clip_fraction        | 0.137       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.56        |
|    explained_variance   | -0.16       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0164     |
|    n_updates            | 2420        |
|    policy_gradient_loss | 0.00708     |
|    std                  | 0.000932    |
|    value_loss           | 6.08e-08    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1136        |
|    iterations           | 244         |
|    time_elapsed         | 439   

# 3. Testen, Speichern und Laden des trainierten Modells

In [1]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

NameError: name 'evaluate_policy' is not defined

In [22]:
model.save("PPO_model_MountainCar")

In [23]:
del model

In [3]:
model = PPO.load("PPO_model_MountainCar", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

In [ ]:
#Andere Methode der Evaluation als Endlosschleife für Dauerdemos
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()